In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_url = '/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv'
test_url = '/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv'
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

In [ ]:
train.head(3)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
X = train.drop('label', axis=1).astype('float32')
y = train['label']

X_test = test.drop('label', axis=1).astype('float32')
y_test = test['label']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,5))
sns.countplot(y, palette=sns.color_palette())
plt.xlabel("Alphabet",fontsize=20)
plt.ylabel("Count", fontsize=20)
plt.title("Images per Alphabet", fontsize=25)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from keras import Sequential,layers,callbacks,preprocessing
X = X.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
y = keras.utils.to_categorical(y- 1, 24)             #to_categorically() labels signs 0-23

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=7, shuffle=True)

In [ ]:
model = Sequential([
    layers.experimental.preprocessing.Rescaling(scale=1.0/255.0, input_shape=(28,28,1)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5,5), (2,2), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (5,5), (2,2), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(100, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(24, activation='softmax')
])
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
model.summary()

In [ ]:
train_datagen = preprocessing.image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                                                       height_shift_range=0.1, shear_range=0.1,
                                                       zoom_range=0.1)

train_datagen.fit(X)

In [ ]:
batch_size = 64
epochs = 15
steps = X_train.shape[0]//batch_size
history = dict()
flow = train_datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)

In [ ]:
class cbs(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if logs['val_loss']<=1e-3:
            print("\n\nValidation loss below 1e-3\n\n")
            self.model.stop_training = True
cb = cbs()

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1.0e-10)
model_hist = model.fit_generator(flow, validation_data=(X_valid, y_valid), verbose=1, shuffle=True,
                               steps_per_epoch=steps, epochs=epochs, callbacks=[cb, reduce_lr])

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(model_hist.history['loss'],'r*-')
plt.plot(model_hist.history['val_loss'],'b*-')
plt.title('Training and Validation Losses')
plt.legend(labels=['Loss','Validation Loss'])
plt.subplot(1,2,2)
plt.plot(model_hist.history['accuracy'],'mo-')
plt.plot(model_hist.history['val_accuracy'],'co-')
plt.title('Training and Validation Accuracy')
plt.legend(labels=['Accuracy','Validation Accuracy'])
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

print("Keras Categorical Accuracy: "+str(model.evaluate(X_test, keras.utils.to_categorical(y_test - 1, 24), verbose=0)[1]))
print("Sklearn Accuracy: "+str(accuracy_score(y_test-1,model.predict_classes(X_test))))